# Predicting secondary protein structure using deep learning

In [13]:
# Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

#import libraries and calculate hyperparameters about train-data dimensions
from keras.models import Sequential
from keras.layers import Embedding, Dense, TimeDistributed, Bidirectional, GRU, LSTM , RNN , Flatten

from keras import backend  as K
import tensorflow as tf


In [6]:
#data preprocessing and anlysis

df = pd.read_csv('datasets/protein-data.csv')
df.head()

#dropping duplicates
df = df[['seq','len','sst3']][(df['len']<40)&(df['has_nonstd_aa']==False)]
df.drop_duplicates(subset='seq',inplace=True)

# clearing the asterisks
def clear_asterisks(df, seqs_column_name, pct=30):
    indices = []
    for i, seq in enumerate(df[seqs_column_name]):
        if (seq.count('*')*100/len(seq)) <= pct: indices.append(i)
    return df.iloc[indices]

df = clear_asterisks(df,'seq')
df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 5425 entries, 0 to 20065
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   seq     5425 non-null   object
 1   len     5425 non-null   int64 
 2   sst3    5425 non-null   object
dtypes: int64(1), object(2)
memory usage: 169.5+ KB


In [22]:
# Preparing the sequences for classification

def ngrams(seq,n=3):
    return ([seq[i:i+n] for i in range(len(seq)-n+1)])

df['seqss']=df['seq'].apply(ngrams)
df.head()


maxlen = df['len'].max()


# tokenizing the sequences
tok_x = Tokenizer(lower=False)
tok_x.fit_on_texts(df['seqss'].values)
x = tok_x.texts_to_sequences(df['seqss'].values)
x = sequence.pad_sequences(x, maxlen=maxlen, padding='post')

tok_y = Tokenizer(char_level=True)
tok_y.fit_on_texts(df['sst3'].values)
y = tok_y.texts_to_sequences(df['sst3'].values)
y = sequence.pad_sequences(y, maxlen=maxlen, padding='post')
y = to_categorical(y)

x.shape, y.shape

((5425, 39), (5425, 39, 4))

In [23]:
#spliting the data

x_tr, x_ts, y_tr, y_ts = train_test_split(x,y,test_size=0.3, random_state=33)
x_tr.shape, y_tr.shape, x_ts.shape, y_ts.shape

n_ngrams = len(tok_x.word_index) + 1
n_tags = len(tok_y.word_index) + 1

In [31]:
# building and running a GRU

gru = Sequential()
gru.add(Embedding(input_dim=n_ngrams,output_dim=x.shape[0]//12,input_length=maxlen))
gru.add(Bidirectional(GRU(x.shape[0]//24,return_sequences=True, recurrent_dropout=0.1)))
gru.add(TimeDistributed(Dense(n_tags,input_dim=y.shape[2],activation='softmax')))
gru.summary()

#running the gru
gru.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
gru.fit(x_tr,y_tr,epochs=15,validation_split=0.1,verbose=1)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 39, 452)           3466840   
                                                                 
 bidirectional (Bidirectiona  (None, 39, 452)          922080    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 39, 4)            1812      
 ibuted)                                                         
                                                                 
Total params: 4,390,732
Trainable params: 4,390,732
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
107/107 [==============================] - 66s 577ms/step - loss: 0.3945 - acc: 0.8441 - val_loss: 0.3060 - val_acc: 0.8771
Epoch 2/15
 19/107 [====>...............

KeyboardInterrupt: 

In [ ]:
# model evaluation

evals = gru.evaluate(x_ts,y_ts)
print('accuracy:', evals[1])

In [38]:
#building and running a RNN 

# Convert the text data to a numerical representation
# x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=500)
# x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=500)

# Define the neural network architecture
model = Sequential()
model.add(Embedding(input_dim=n_ngrams, output_dim=x.shape[0]//12, input_length=maxlen))
model.add(Dense(64, activation='relu'))
model.add(Dense(n_tags ,input_dim=y.shape[2],activation='softmax'))

print(n_tags , x.shape[0]//12 , maxlen)

# Compile the neural network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the neural network
model.fit(x_tr, y_tr, epochs=250)

# Evaluate the neural network
y_pred = model.predict(x_ts)
accuracy = tf.keras.metrics.binary_accuracy(y_ts, y_pred)
print('Accuracy:', accuracy.numpy())

4 452 39
Epoch 1/250
119/119 [==============================] - 6s 43ms/step - loss: 0.7149 - accuracy: 0.7687
Epoch 2/250
119/119 [==============================] - 5s 44ms/step - loss: 0.5071 - accuracy: 0.8089
Epoch 3/250
119/119 [==============================] - 5s 44ms/step - loss: 0.4856 - accuracy: 0.8136
Epoch 4/250
119/119 [==============================] - 6s 52ms/step - loss: 0.4781 - accuracy: 0.8149
Epoch 5/250
119/119 [==============================] - 6s 47ms/step - loss: 0.4734 - accuracy: 0.8168
Epoch 6/250
119/119 [==============================] - 5s 45ms/step - loss: 0.4707 - accuracy: 0.8174
Epoch 7/250
119/119 [==============================] - 6s 46ms/step - loss: 0.4686 - accuracy: 0.8180
Epoch 8/250
119/119 [==============================] - 5s 44ms/step - loss: 0.4671 - accuracy: 0.8184
Epoch 9/250
119/119 [==============================] - 6s 47ms/step - loss: 0.4665 - accuracy: 0.8185
Epoch 10/250
119/119 [==============================] - 6s 47ms/step - lo

KeyboardInterrupt: 